In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
import os

# Specify the path to the zip file and the directory to extract to
zip_file_path = '/content/drive/MyDrive/FaceMask detection/archive (1).zip'
extract_to_path = '/content/drive/MyDrive/FaceMask detection/'

# Ensure the extraction directory exists
os.makedirs(extract_to_path, exist_ok=True)

# Open the zip file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Files have been extracted to {extract_to_path}")


Files have been extracted to /content/drive/MyDrive/FaceMask detection/


In [2]:
pip install  tensorflow opencv-python matplotlib scikit-learn

In [5]:
import os
import numpy as np
import  matplotlib.pyplot as plt
import cv2
from  sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Load and Preprocess the Dataset**

In [6]:
dataset_path='/content/drive/MyDrive/FaceMask detection/data'

In [7]:
data=[]
lables=[]
categories=['with_mask','without_mask']

In [8]:
for category in categories:
  folder_path=os.path.join(dataset_path,category)
  label=categories.index(category)
  for img_name in os.listdir(folder_path):
    img_path=os.path.join(folder_path,img_name)
    try:
      img=cv2.imread(img_path)
      img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      img=cv2.resize(img,(128,128))
      data.append(img)
      lables.append(label)
    except Exception as e:
      print(e)

In [9]:
#convert data in numpy arrays
data=np.array(data)/255.0
lables=np.array(lables)

In [10]:
#split data into train and test
x_train,x_test,y_train,y_test=train_test_split(data,lables,test_size=0.2,random_state=42)

In [11]:
from ast import Num
#use One-hot encodings for lables
y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

**Build the Cnn model**

In [12]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')



]



)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,578 (28.20 MB)

 Trainable params: 7,392,578 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Model**

In [15]:
# data Augmentation
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [16]:
histroy=model.fit(datagen.flow(x_train,y_train,batch_size=32),validation_data=(x_test,y_test),epochs=30)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


115/115 ━━━━━━━━━━━━━━━━━━━━ 29s 184ms/step - accuracy: 0.7625 - loss: 1.4337 - val_accuracy: 0.8456 - val_loss: 0.3725
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.8505 - loss: 0.3032 - val_accuracy: 0.9332 - val_loss: 0.2377
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accuracy: 0.9029 - loss: 0.2727 - val_accuracy: 0.9332 - val_loss: 0.2270
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 22s 145ms/step - accuracy: 0.9200 - loss: 0.2434 - val_accuracy: 0.9266 - val_loss: 0.2335
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 139ms/step - accuracy: 0.9161 - loss: 0.2307 - val_accuracy: 0.9441 - val_loss: 0.1748
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 16s 130ms/step - accuracy: 0.9273 - loss: 0.2038 - val_accuracy: 0.9485 - val_loss: 0.1787
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 15s 128ms/step - accuracy: 0.9218 - loss: 0.2133 - val_accuracy: 0.9463 - val_loss: 0.2010
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.9285 - loss: 0.2193 - val

In [17]:
loss,accuracy=model.evaluate(x_test,y_test)
print(f"Test Accuracy:{accuracy*100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9516 - loss: 0.1371
Test Accuracy:95.40%


In [18]:
from tensorflow.keras.models import Sequential

# Assume `model` is your trained CNN model
model = Sequential()

# Save the entire model to a file
model.save("FMDetection.h5")
print("Model saved as cnn_model.h5")

# To load the model later
from tensorflow.keras.models import load_model
loaded_model = load_model("FMDetection.h5")
print("Model loaded successfully")


Model saved as cnn_model.h5
Model loaded successfully
